# 1. Работа с изображениями.

In [1]:
from PIL import Image, ImageFilter
import math
import numpy as np
import PIL

In [2]:
h = 200
w = 200
blackImg = np.zeros((h, w), dtype=np.uint8)
whiteImg = np.full((h, w), 255, dtype=np.uint8)
img = PIL.Image.fromarray(blackImg, 'L')
img.save("bImg.png")

In [3]:
img = PIL.Image.fromarray(whiteImg, 'L')
img.save("wImg.png")

In [4]:
image = np.zeros((h, w, 3), dtype=np.uint8)
for i in range(200):
    for j in range(200):
        image[i, j, 0] = 255
img = PIL.Image.fromarray(image, 'RGB')
img.save("image1.png")

In [5]:
image = np.zeros((h, w, 3), dtype=np.uint8)
for i in range(200):
    for j in range(200):
        image[i, j] = (i+j) % 256
img = PIL.Image.fromarray(image, 'RGB')
img.save("image2.png")

# 3. Отрисовка прямых линий

Реализовать все описанные в лекциях алгоритмы отрисовки прямых (до алгоритма Брезенхема включительно).

Для каждого алгоритма сохранить в файл изображение размера 200х200 с нарисованной на нём «звездой» (см. лекции).


In [6]:
def line(x0, y0, x1, y1, image, color):
    steep = False
    # Если смещение по оси X меньше смещения по оси Y , меняем местами координаты.
    if (abs(x0-x1) < abs(y0-y1)):
        x0, y0 = y0, x0
        x1, y1 = y1, x1
        steep = True
    # Если начало прямой находится правее конца, меняем начало и конец местами.
    if(x0 > x1):
        x0, x1 = x1, x0
        y0, y1 = y1, y0

    dx = abs(x1 - x0)
    dy = y1 - y0
    derror = 2 * abs(dy)
    error = 0.0
    y = y0

    for x in range(x0, x1+1):
        if(steep):
            image[y, x] = color
        else:
            image[x, y] = color
        error += derror
        if(error > dx):
            y += (1 if (y1 > y0) else -1)
            error -= 2*dx
    return image


image = np.zeros((h, w), dtype=np.uint8)
x0, y0 = 100, 100
for i in range(0, 13):
    a = 2 * i * math.pi / 13
    x1 = int(100 + 95 * math.cos(a))
    y1 = int(100 + 95 * math.sin(a))
    img = PIL.Image.fromarray(line(x0, y0, x1, y1, image, 255-i), 'L')
img.save("image3.png")

# 4. Работа с трёхмерной моделью (вершины)
Создать класс, содержащий трёхмерную модель в виде массив трёхмерных координат точек (для может потребоваться создать класс трёхмерных координат). 
Считать из приложенного файла obj строки, содержащие информацию о вершинах модели в объект созданного класса:


In [7]:
def parseV():
    try:
        with open('fox.obj', 'r') as f:
            data = f.readlines()
    except IOError:
        print('IOError')
    lst = [str[2:].split() for str in data[4:] if 'v ' in str]
    lst = [list(map(float, str)) for str in lst]
    return lst
v = parseV()
v

[[10.013863, 57.761101, 8.092894],
 [11.597156, 30.54607, -26.744658],
 [11.53214, 49.64645, -24.654264],
 [7.041744, 42.900967, -37.667263],
 [10.698391, 49.27322, -37.793137],
 [4.21216, 43.85033, -40.912064],
 [9.038975, 16.989401, 15.505488],
 [11.207468, 53.822937, 21.917948],
 [10.067627, 60.659023, 22.694967],
 [6.766037, 37.404408, 27.425133],
 [7.787885, 66.50647, 32.700138],
 [4.684775, 72.120506, 41.261551],
 [10.391252, 64.281273, 40.747532],
 [6.704249, 53.417492, 52.455326],
 [4.759357, 69.991417, 51.111191],
 [9.525023, 66.221077, 48.920918],
 [2.446671, 62.23999, 57.674969],
 [5.348628, 57.736511, 58.43856],
 [1.544795, 55.042629, 66.616386],
 [3.744807, 52.667049, 61.826649],
 [10.111939, 35.768944, 10.992324],
 [9.993451, 37.712261, -16.556917],
 [10.013863, 42.553417, 26.717955],
 [9.988014, 56.955791, 42.201694],
 [10.013863, 58.609661, 51.223545],
 [12.592718, 78.907196, 50.617199],
 [5.973053, 34.281334, -44.943275],
 [7.032498, 48.605991, -40.948818],
 [9.982605,

# 5. Отрисовка вершин трёхмерной модели
Нарисовать вершины модели (игнорируя координату Z) на изображении размером (1000, 1000).


In [8]:
image = np.zeros((1000, 1000), dtype=np.uint8)
for i in range(0, len(v)-1):
    img = PIL.Image.fromarray(line(-int(v[i][1]) + 500, int(v[i][2] + 500),
                                   -int(v[i][1]) + 500, int(v[i][2]) + 500, image, 255), 'L')
img.save("image5.png")

# 6. Работа с трёхмерной моделью (полигоны)
Считать из приложенного файла строки, содержащие информацию о полигонах модели.
Сведения о полигонах в файле хранятся в формате:
f v1/vt1/vn1 v2/vt2/vn2 v3/vt3/vn3
В рамках лабораторной загрузить в память необходимо только первые значения в каждой тройке – номера вершин, загруженных ранее.
Обратите внимание, что вершины нумеруются, начиная с единицы.


In [9]:
def parseF():
    try:
        with open('fox.obj', 'r') as f:
            data = f.readlines()
    except IOError:
        print('IOError')
    lst = [str[2:].split() for str in data[4:] if 'f ' in str]
    nlst = [j.split('/') for str in lst for j in str]
    lst = [list(map(int, str))[0] for str in nlst]
    return lst
f = parseF()
len(v)

290

# 7. Отрисовка рёбер трёхмерной модели
Отрисовать все рёбра всех полигонов модели с помощью алгоритма Брезенхема (координаты вершин округляем до ближайшего целого).


In [10]:
image = np.zeros((1000, 1000), dtype=np.uint8)
k = 5
for i in range(0, len(f)//3-1):
    img = PIL.Image.fromarray(line(-int(v[f[3*i]-1][1]) * k + 500, int(v[f[3*i]-1][2]) * k + 500,
                                   -int(v[f[3*i+1]-1][1]) * k + 500, int(v[f[3*i+1]-1][2]) * k + 500, image, 255), 'L')
    img = PIL.Image.fromarray(line(-int(v[f[3*i]-1][1]) * k + 500, int(v[f[3*i]-1][2]) * k + 500,
                                   -int(v[f[3*i+2]-1][1]) * k + 500, int(v[f[3*i+2]-1][2]) * k + 500, image, 255), 'L')
    img = PIL.Image.fromarray(line(-int(v[f[3*i+1]-1][1]) * k + 500, int(v[f[3*i+1]-1][2]) * k + 500,
                                   -int(v[f[3*i+2]-1][1]) * k + 500, int(v[f[3*i+2]-1][2]) * k + 500, image, 255), 'L')
img.save("image6.png")